<a href="https://colab.research.google.com/github/Kimjibeom/Python_project/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_TOP100_%ED%81%AC%EB%A1%A4%EB%A7%81%26%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

링크 수집을 위한 라이브러리 설치

In [6]:
!pip install selenium

     |████████████████████████████████| 911kB 12.9MB/s 


라이브러리 import

In [7]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [8]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.n

분석할 데이터 모으기(크롤링)

네이버 영화 TOP 100 페이지 별 링크 수집

In [13]:
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page= 일간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page= 주간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page= 월간

#크롤링을 위한 환경 설정
def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] # 100개의 영화 링크를 수집할 리스트 변수 선언
  return_links = []

  for pageNo in range(startPageNo -1, lastPageNo) :
    # 실시간 TOP 100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=" +str(pageNo +1)

    req = requests.get(url)     # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml')  # html 파싱을 위한 객체 생성
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href="">

    for movielink in movielinks:
      link = str(movielink.get('href'))
      links.append("https://series.naver.com"+link)  # 접속할 수 있는 전체 링크 형태로 변환해서 append

  return links




네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [14]:
def getMovieDataFromNaverSeries(links):
  title_infos = []    # 제목
  content_infos =[]   # 줄거리
  genre_infos = []    # 장르
  score_infos = []    # 평점
  date_infos = []     # 개봉일

  url2 = "https://www.naver.com"

  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url2)
  time.sleep(3.0)   # 3초 sleep

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + "t")

  for link in links:
    print(link + ' 수집 중.........')
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새창 활성화 
    time.sleep(3.0)

    html_source = driver.page_source  # req.text 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년관람불가 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = "".join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버' :
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score) # 정수형으로 변경
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르','')
      genre = genre.replace('\n','')
      genre = genre.replace('\t','')
      genre_infos.append(genre)
      print(genre)

      # 제목, 줄거리

      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta", {"property":"og:title"}).get('content')
      title_infos.append(title)
      print(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0:
        content_infos.append("줄거리 오류")

      else:
        for contents in contents_texts:
          # 줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace('\r','')    # 줄 바꿈 제거
          temp = temp.replace('\xa0','')  # 공백 제거
          content_infos.append(temp)

    elif newflag == '' :
      print('청불 영화로 데이터 수집하지 않습니다.')

  print('수집 완료 합니다.....')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {"제목":title_infos, "평점":score_infos,"장르":genre_infos,"줄거리":content_infos}

  # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates("줄거리", keep='first') # 줄거리 비교해서 중복된 영화를 삭제 (첫 범째 수집 영화만 남김)

  return movie_df2

csv 파일로 저장

In [15]:
def dftoCsv(movie_df, num):
    try:
      movie_df.to_csv(('movie_my_data_'+ str(num) +'.csv'),
                      sep=',', na_rep='NaN', encoding='euc-kr')
      
    except :
      print("Error")

크롤링 실행

In [16]:
# 크롤링 할 링크 수집
links = getPageLinksWantRange(1,5)

# 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

# csv 파일 저장
dftoCsv(movie_df2, len(movie_df2 ))

https://series.naver.com/movie/detail.nhn?productNo=3400212 수집 중.........
7
공포/스릴러
콰이어트 플레이스
https://series.naver.com/movie/detail.nhn?productNo=6291892 수집 중.........
9
애니메이션
극장판 귀멸의 칼날: 무한열차편
https://series.naver.com/movie/detail.nhn?productNo=6315371 수집 중.........
9
드라마
크루엘라
https://series.naver.com/movie/detail.nhn?productNo=3024534 수집 중.........
8
액션
킬러의 보디가드
https://series.naver.com/movie/detail.nhn?productNo=3639460 수집 중.........
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6119777 수집 중.........
7
드라마
미나리
https://series.naver.com/movie/detail.nhn?productNo=2822635 수집 중.........
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6283368 수집 중.........
6
액션
파이프라인
https://series.naver.com/movie/detail.nhn?productNo=6243813 수집 중.........
9
액션
크루즈 패밀리: 뉴 에이지
https://series.naver.com/movie/detail.nhn?productNo=6257268 수집 중.........
9
액션
크루즈 패밀리: 뉴 에이지
https://series.naver.com/movie/detail.nhn?productNo=6023769 수집 중.........
9
애니메이션
소울


 라이브러리 임포트

In [20]:
import plotly.express as px
import plotly.express as go
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# 크롤링한 데이터 불러오기
df = pd.read_csv('/content/movie_my_data_66.csv', encoding='euc-kr')
df.head(5)

# 영화 별 평점 시각화
fig = px.scatter(df, x="제목", y='평점', color='장르')
fig.show()

# 평점이 9점인 영화 분류하여 df_map 변수에 담기
df_map = df[df.평점==9]
df_map.tail(5)

# 9점 평점 장르 시각화
fig = px.pie(df_map, values='평점', names='장르')
fig.show()

월간 TOP 100 영화 데이터 시각화

In [21]:
# 월간 TOP 100 영화 데이터 불러오기
df4 = pd.read_csv('/content/movie_my_data_monthly_64.csv', encoding='euc-kr')
df4.head(5)

# 월간 TOP 100 영화 별 평점 시각화(scatter)
fig = px.scatter(df4, x="제목", y='평점', color='장르')
fig.show()

# 월간 TOP 100 영화 별 평점 시각화(bar)
fig = px.bar(df4, x="제목", y='평점', color='장르')
fig.show()

# 월간 TOP 100 장르별 평점 시각화
fig = px.pie(df4, values='평점', names='장르')
fig.show()